# AutoKeras

* https://autokeras.com/

## Imports and Global Settings

In [1]:
import sys
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split

sys.path.append('../')
from passkeys import RDS_ENDPOINT, RDS_PASSWORD

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

## Database Connection

In [2]:
username = 'postgres'
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

### Datasets

In [3]:
df = pd.read_sql_table('model_training_data', connection)

### Restrict to previous years games only

In [4]:
df = df[df['league_year_end'] != 23]

In [5]:
df.sort_values('game_id', ascending=False).head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consensus_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_vla,offrtg_vla_std,defrtg_rank,defrtg_vla,defrtg_vla_std,netrtg_rank,netrtg_vla,netrtg_vla_std,ast_pct_rank,ast_pct_vla,ast_pct_vla_std,ast_v_tov_rank,ast_v_tov_vla,ast_v_tov_vla_std,ast_ratio_rank,ast_ratio_vla,...,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,opp_fg_pct_atb3_rank,opp_fg_pct_atb3_vla,opp_fg_pct_atb3_vla_std

## Basic Data Overview

In [6]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 0 to 9328
Data columns (total 1223 columns):
 #     Column                                 Non-Null Count  Dtype  
---    ------                                 --------------  -----  
 0     game_id                                9200 non-null   object 
 1     CLS_TARGET_home_margin_GT_home_spread  9200 non-null   bool   
 2     REG_TARGET_actual_home_margin          9200 non-null   float64
 3     home_team_num                          9200 non-null   int64  
 4     away_team_num                          9200 non-null   int64  
 5     league_year_end                        9200 non-null   int64  
 6     fd_line_home                           15 non-null     float64
 7     dk_line_home                           14 non-null     float64
 8     covers_consensus_home                  15 non-null     float64
 9     home_spread                            9200 non-null   float64
 10    gp                                     9171 

In [7]:
df.head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consensus_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_vla,offrtg_vla_std,defrtg_rank,defrtg_vla,defrtg_vla_std,netrtg_rank,netrtg_vla,netrtg_vla_std,ast_pct_rank,ast_pct_vla,ast_pct_vla_std,ast_v_tov_rank,ast_v_tov_vla,ast_v_tov_vla_std,ast_ratio_rank,ast_ratio_vla,...,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,opp_fg_pct_atb3_rank,opp_fg_pct_atb3_vla,opp_fg_pct_atb3_vla_std

## AutoKeras - Regression

* https://autokeras.com/tutorial/structured_data_regression/

### Setup and Preprocessing

In [8]:
target = ['REG_TARGET_actual_home_margin']
drop_features = ['fd_line_home', 'dk_line_home', 'covers_consensus_home', 'game_id',
                 'REG_TARGET_actual_home_margin', 'CLS_TARGET_home_margin_GT_home_spread']
main_features = ['home_team_num', 'away_team_num', 'home_spread',
                 'league_year_end', 'day_of_season', 'elo1_pre',
                 'elo2_pre', 'elo_prob1', 'elo_prob2']
rank_features = [feature for feature in list(df) if 'rank' in feature]
zscore_features = [feature for feature in list(df) if 'zscore' in feature]
                 
other_features = [feature for feature in list(df) if feature not in target + main_features + drop_features]

features_to_use = target + main_features

In [9]:
reg_df = df[features_to_use]

In [10]:
reg_df.head()

,REG_TARGET_actual_home_margin,home_team_num,away_team_num,home_spread,league_year_end,day_of_season,elo1_pre,elo2_pre,elo_prob1,elo_prob2,gp_rank,win_rank,loss_rank,w_pct_rank,mins_rank,pts_rank,fgm_rank,fga_rank,fg_pct_rank,fg3m_rank,fg3a_rank,fg3_pct_rank,ftm_rank,fta_rank,ft_pct_rank,oreb_rank,dreb_rank,reb_rank,ast_rank,tov_rank,stl_rank,blk_rank,blka_rank,pf_rank,pfd_rank,p_m_rank,gp_rank_opp,win_rank_opp,loss_rank_opp,w_pct_rank_opp,mins_rank_opp,pts_rank_opp,fgm_rank_opp,fga_rank_opp,fg_pct_rank_opp,fg3m_rank_opp,fg3a_rank_opp,fg3_pct_rank_opp,ftm_rank_opp,fta_rank_opp,ft_pct_rank_opp,oreb_rank_opp,dreb_rank_opp,reb_rank_opp,ast_rank_opp,tov_rank_opp,stl_rank_opp,blk_rank_opp,blka_rank_opp,pf_rank_opp,pfd_rank_opp,p_m_rank_opp,offrtg_rank,defrtg_rank,netrtg_rank,ast_pct_rank,ast_v_tov_rank,ast_ratio_rank,oreb_pct_rank,dreb_pct_rank,reb_pct_rank,tov_pct_rank,efg_pct_rank,ts_pct_rank,pace_rank,pie_rank,poss_rank,offrtg_rank_opp,defrtg_rank_opp,netrtg_rank_opp,ast_pct_rank_opp,ast_v_tov_rank_opp,ast_ratio_rank_opp,oreb_pct_rank_opp,dreb_pct_rank_opp,reb_pct_rank_opp,tov_pct_rank_opp,efg_pct_rank_opp,ts_pct_rank_opp,pace_rank_opp,pie_rank_opp,poss_rank_opp,fta_rate_rank,opp_efg_pct_rank,opp_fta_rate_rank,opp_tov_pct_rank,opp_oreb_pct_rank,fta_rate_rank_opp,opp_efg_pct_rank_opp,opp_fta_rate_rank_opp,opp_tov_pct_rank_opp,opp_oreb_pct_rank_opp,pts_off_tov_rank,second_pts_rank,fbps_rank,pitp_rank,opp_pts_off_tov_rank,opp_second_pts_rank,opp_fbps_rank,opp_pitp_rank,pts_off_tov_rank_opp,second_pts_rank_opp,fbps_rank_opp,pitp_rank_opp,opp_pts_off_tov_rank_opp,opp_second_pts_rank_opp,opp_fbps_rank_opp,opp_pitp_rank_opp,pct_fga_2pt_rank,pct_fga_3pt_rank,pct_pts_2pt_rank,pct_pts_2pt_mid_rank,pct_pts_3pt_rank,pct_pts_fbps_rank,pct_pts_ft_rank,pct_pts_off_tov_rank,pct_pts_pitp_rank,pct_ast_2fgm_rank,pct_uast_2fgm_rank,pct_ast_3fgm_rank,pct_uast_3fgm_rank,pct_ast_fgm_rank,pct_uast_fgm_rank,pct_fga_2pt_rank_opp,pct_fga_3pt_rank_opp,pct_pts_2pt_rank_opp,pct_pts_2pt_mid_rank_opp,pct_pts_3pt_rank_opp,pct_pts_fbps_rank_opp,pct_pts_ft_rank_opp,pct_pts_off_tov_rank_opp,pct_pts_pitp_rank_opp,pct_ast_2fgm_rank_opp,pct_uast_2fgm_rank_opp,pct_ast_3fgm_rank_opp,pct_uast_3fgm_rank_opp,pct_ast_fgm_rank_opp,pct_uast_fgm_rank_opp,opp_fgm_rank,opp_fga_rank,opp_fg_pct_rank,opp_3pm_rank,opp_3pa_rank,opp_3pt_pct_rank,opp_ftm_rank,opp_fta_rank,opp_ft_pct_rank,opp_oreb_rank,opp_dreb_rank,opp_reb_rank,opp_ast_rank,opp_tov_rank,opp_stl_rank,opp_blk_rank,opp_blka_rank,opp_pf_rank,opp_pfd_rank,opp_pts_rank,opp_pm_rank,opp_fgm_rank_opp,opp_fga_rank_opp,opp_fg_pct_rank_opp,opp_3pm_rank_opp,opp_3pa_rank_opp,opp_3pt_pct_rank_opp,opp_ftm_rank_opp,opp_fta_rank_opp,opp_ft_pct_rank_opp,opp_oreb_rank_opp,opp_dreb_rank_opp,opp_reb_rank_opp,opp_ast_rank_opp,opp_tov_rank_opp,opp_stl_rank_opp,opp_blk_rank_opp,opp_blka_rank_opp,opp_pf_rank_opp,opp_pfd_rank_opp,opp_pts_rank_opp,opp_pm_rank_opp,dist_feet_rank,dist_miles_rank,dist_miles_off_rank,dist_miles_def_rank,avg_speed_rank,avg_speed_off_rank,avg_speed_def_rank,dist_feet_rank_opp,dist_miles_rank_opp,dist_miles_off_rank_opp,dist_miles_def_rank_opp,avg_speed_rank_opp,avg_speed_off_rank_opp,avg_speed_def_rank_opp,fgm_ra_rank,fga_ra_rank,fg_pct_ra_rank,fgm_paint_rank,fga_paint_rank,fg_pct_paint_rank,fgm_mr_rank,fga_mr_rank,fg_pct_mr_rank,fgm_lc3_rank,fga_lc3_rank,fg_pct_lc3_rank,fgm_rc3_rank,fga_rc3_rank,fg_pct_rc3_rank,fgm_c3_rank,fga_c3_rank,fg_pct_c3_rank,fgm_atb3_rank,fga_atb3_rank,fg_pct_atb3_rank,fgm_ra_rank_opp,fga_ra_rank_opp,fg_pct_ra_rank_opp,fgm_paint_rank_opp,fga_paint_rank_opp,fg_pct_paint_rank_opp,fgm_mr_rank_opp,fga_mr_rank_opp,fg_pct_mr_rank_opp,fgm_lc3_rank_opp,fga_lc3_rank_opp,fg_pct_lc3_rank_opp,fgm_rc3_rank_opp,fga_rc3_rank_opp,fg_pct_rc3_rank_opp,fgm_c3_rank_opp,fga_c3_rank_opp,fg_pct_c3_rank_opp,fgm_atb3_rank_opp,fga_atb3_rank_opp,fg_pct_atb3_rank_opp,opp_fgm_ra_rank,opp_fga_ra_rank,opp_fg_pct_ra_rank,opp_fgm_paint_rank,...,offrtg_vla_std,defrtg_vla_std,netrtg_vla_std,ast_pct_vla_std,ast_v_tov_vla_std,as

In [12]:
reg_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 0 to 9328
Data columns (total 614 columns):
 #    Column                              Dtype  
---   ------                              -----  
 0    REG_TARGET_actual_home_margin       float64
 1    home_team_num                       int64  
 2    away_team_num                       int64  
 3    home_spread                         float64
 4    league_year_end                     int64  
 5    day_of_season                       int64  
 6    elo1_pre                            float64
 7    elo2_pre                            float64
 8    elo_prob1                           float64
 9    elo_prob2                           float64
 10   gp_rank                             float64
 11   win_rank                            float64
 12   loss_rank                           float64
 13   w_pct_rank                          float64
 14   mins_rank                           float64
 15   pts_rank                            

In [13]:
def split_data(df, target, test_size=0.3, random_state=0):
    """Deep Learning specific wrapper for normal sklearn train/test split.

    Args:
        df (Pandas Dataframe)
        target (str): Name of target column.
        test_size (float, optional): Defaults to 0.3.
        random_state (int, optional): Defaults to 0.

    Returns:
        Tuple of Dataframes: X_train, X_test, y_train, y_test
    """
    y = df[target]
    X = df.drop(columns=target)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state)
    print(f"X_train Shape: {X_train.shape}")
    print(f"X_test Shape: {X_test.shape}")
    print(f"y_train Shape: {y_train.shape}")
    print(f"y_test Shape: {y_test.shape}")
    return X_train, X_test, y_train, y_test

In [14]:
cls_target = 'CLS_TARGET_home_margin_GT_home_spread'
reg_target = 'REG_TARGET_actual_home_margin'

In [15]:
X_train, X_test, y_train, y_test = split_data(reg_df, reg_target)

X_train Shape: (6440, 613)
X_test Shape: (2760, 613)
y_train Shape: (6440,)
y_test Shape: (2760,)


### Fit

In [16]:
reg = ak.StructuredDataRegressor(max_trials=20, overwrite=True)

In [17]:
reg.fit(X_train, y_train)

Trial 20 Complete [00h 04m 51s]
val_loss: 178.087890625

Best val_loss So Far: 175.84593200683594
Total elapsed time: 01h 06m 39s
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
202/202 [==============================] - 10s 14ms/step - loss: 183.7609 - mean_squared_error: 183.7609
Epoch 2/5
202/202 [==============================] - 3s 13ms/step - loss: 168.6087 - mean_squared_error: 168.6087
Epoch 3/5
202/202 [==============================] - 3s 13ms/step - loss: 163.6568 - mean_squared_error: 163.6568
Epoch 4/5
202/202 [==============================] - 3s 14ms/step - loss: 161.3755 - mean_squared_error: 161.3755
Epoch 5/5
202/202 [==============================] - 3s 13ms/step - loss: 157.3091 - mean_squared_error: 157.3091
INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


### Predict

In [18]:
predicted_y = reg.predict(X_test)

87/87 [==============================] - 1s 14ms/step


### Evaluate

In [19]:
print(reg.evaluate(X_test, y_test))

87/87 [==============================] - 5s 14ms/step - loss: 162.8011 - mean_squared_error: 162.8011
[162.8011474609375, 162.8011474609375]


### Model Finalization and Storage

In [20]:
final_reg_model = reg.export_model()

In [21]:
final_reg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 613)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 613)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 613)              1227      
 n)                                                              
                                                                 
 dense (Dense)               (None, 512)               314368    
                                                                 
 re_lu (ReLU)                (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0     

In [22]:
print(type(final_reg_model))

<class 'keras.engine.functional.Functional'>


In [23]:
# try:
#     final_reg_model.save('../models/AutoKeras/AK_REG_Main_Features', save_format="tf")
# except Exception:
#     final_reg_model.save('../models/AutoKeras/AK_REG_Main_Features.h5')

INFO:tensorflow:Assets written to: ../models/AutoKeras/vlastd_rank_AK_REG/assets


## Classification

### Setup and Preprocessing

In [24]:
target = ['CLS_TARGET_home_margin_GT_home_spread']
drop_features = ['fd_line_home', 'dk_line_home', 'covers_consensus_home', 'game_id',
                 'REG_TARGET_actual_home_margin', 'CLS_TARGET_home_margin_GT_home_spread']
main_features = ['home_team_num', 'away_team_num', 'home_spread',
                 'league_year_end', 'day_of_season', 'elo1_pre',
                 'elo2_pre', 'elo_prob1', 'elo_prob2']
rank_features = [feature for feature in list(df) if 'rank' in feature]
zscore_features = [feature for feature in list(df) if 'zscore' in feature]
                 
other_features = [feature for feature in list(df) if feature not in target + main_features + drop_features]

features_to_use = target + main_features

In [25]:
cls_df = df[features_to_use]

In [26]:
cls_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 0 to 9328
Columns: 614 entries, CLS_TARGET_home_margin_GT_home_spread to contested_shots_vla_std_opp
dtypes: bool(1), float64(609), int64(4)
memory usage: 43.1 MB


In [27]:
X_train, X_test, y_train, y_test = split_data(cls_df, cls_target)

X_train Shape: (6440, 613)
X_test Shape: (2760, 613)
y_train Shape: (6440,)
y_test Shape: (2760,)


### Fit

In [28]:
cls = ak.StructuredDataClassifier(max_trials=20, overwrite=True)

In [29]:
cls.fit(X_train, y_train)

Trial 20 Complete [00h 02m 29s]
val_accuracy: 0.5135350227355957

Best val_accuracy So Far: 0.5406050682067871
Total elapsed time: 00h 53m 38s
INFO:tensorflow:Oracle triggered exit
Epoch 1/4
202/202 [==============================] - 9s 14ms/step - loss: 0.8040 - accuracy: 0.5048
Epoch 2/4
202/202 [==============================] - 3s 13ms/step - loss: 0.6947 - accuracy: 0.5660
Epoch 3/4
202/202 [==============================] - 3s 13ms/step - loss: 0.6563 - accuracy: 0.6140
Epoch 4/4
202/202 [==============================] - 3s 13ms/step - loss: 0.6139 - accuracy: 0.6686
INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


### Predict

In [30]:
predicted_y = cls.predict(X_test)

87/87 [==============================] - 1s 12ms/step


### Evaluate

In [31]:
print(cls.evaluate(X_test, y_test))

87/87 [==============================] - 5s 13ms/step - loss: 0.7427 - accuracy: 0.4971
[0.7427108287811279, 0.49710145592689514]


### Model Finalization and Storage

In [32]:
final_cls_model = cls.export_model()

In [33]:
final_cls_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 613)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 613)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 613)              1227      
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               157184    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                             

In [34]:
print(type(final_cls_model))

<class 'keras.engine.functional.Functional'>


In [35]:
# try:
#     final_cls_model.save('../models/AutoKeras/AK_CLS_Main_Features', save_format="tf")
# except Exception:
#     final_cls_model.save('../models/AutoKeras/AK_CLS_Main_Features.h5')

INFO:tensorflow:Assets written to: ../models/AutoKeras/vlastd_rank_AK_CLS/assets
